In [1]:
#import duckdb
#con = duckdb.connect(database=':memory:')
#con.execute("INSTALL sqlite;")
#con.execute("LOAD sqlite;")
#con.execute("CALL sqlite_attach('data/2022-12-23-ecas-export.db');")
#con.execute("PRAGMA show_tables;")
#con.fetchall()

In [2]:
#con.execute("DESCRIBE android_metadata;")
#con.fetchall()

In [3]:
#import sqlalchemy 

In [4]:
import duckdb
import pandas as pd
import sqlalchemy
# No need to import duckdb_engine
#  SQLAlchemy will auto-detect the driver needed based on your connection string!

# Import ipython-sql Jupyter extension to create SQL cells
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [5]:
%sql duckdb:///:memory:

In [6]:
%%sql

INSTALL sqlite;
LOAD sqlite;

""


In [7]:
%%sql

CALL sqlite_attach('data/2022-12-23-ecas-export.db');
PRAGMA show_tables;

,name
0,android_metadata
1,meter
2,reading


In [8]:
%%sql 

SELECT * FROM meter;

,_id,name,units,comment,vsf,tsf,cost,fcost,invert,vmsetup,type,currency,scaling,phyunits,bidir,prod
0,1,b'Strom',b'',b'',b'50',b'50',b'',b'',b'0',None,b'0',b'$',b'1.0',b'',b'0',b'0'
1,2,b'W\xc3\xa4rmestrom 18.2',b'',b'',b'50',b'50',b'',b'',b'0',None,b'0',b'$',b'1.0',b'',b'0',b'0'
2,3,b'W\xc3\xa4rmestrom 18.1',b'',b'',b'50',b'50',b'',b'',b'0',None,b'0',b'$',b'1.0',b'',b'0',b'0'
3,4,b'Wasser',b'',b'',b'50',b'50',b'',b'',b'0',None,b'0',b'$',b'1.0',b'',b'0',b'0'
4,5,b'L\xc3\xbcftungsanlage',b'',b'',b'50',b'50',b'',b'',b'0',None,b'0',b'$',b'1.0',b'',b'0',b'0'
5,6,b'Waschmaschine',b'',b'',b'50',b'50',b'',b'',b'0',None,b'0',b'$',b'1.0',b'',b'0',b'0'
6,7,b'Geschirrsp\xc3\xbcler',b'',b'',b'50',b'50',b'',b'',b'0',None,b'0',b'$',b'1.0',b'',b'0',b'0'
7,8,b'Offen',b'',b'',b'50',b'50',b'',b'',b'0',None,b'0',b'$',b'1.0',b'',b'0',b'0'
8,9,b'Reset',b'',b'',b'50',b'50',b'',b'',b'0',None,b'0',b'$',b'1.0',b'',b'0',b'0'
9,10,b'Nachladen',b'',b'',b'50',b'50',b'',b'',b'0',None,b'0',b'$',b'1.0',b'',b'0',b'0'


In [9]:
%%sql 

CREATE OR REPLACE VIEW strom AS 
SELECT meterid, decode(date)::DATETIME AS date, decode(value)::INT AS value 
FROM reading WHERE meterid = 1
;

""


In [15]:
%%sql 

SELECT 
  *, 
  date_sub('minute', lag(date) over(order by date), date) AS minutes, 
  value - lag(value) over(order by date) AS consumption,
  24.0 * 60.0 * consumption / minutes AS consumption_per_day
FROM strom
ORDER BY date
;

,meterid,date,value,minutes,consumption,consumption_per_day
0,1,2020-11-30 07:07:00,12162,NaN,NaN,NaN
1,1,2020-11-30 17:35:00,12165,628.0,3.0,6.878981
2,1,2020-11-30 23:59:00,12168,384.0,3.0,11.250000
3,1,2020-12-01 07:18:00,12170,439.0,2.0,6.560364
4,1,2020-12-01 14:23:00,12173,425.0,3.0,10.164706
...,...,...,...,...,...,...
346,1,2022-12-19 07:17:00,18660,1366.0,8.0,8.433382
347,1,2022-12-20 10:55:00,18674,1658.0,14.0,12.159228
348,1,2022-12-21 08:13:00,18682,1278.0,8.0,9.014085
349,1,2022-12-22 07:31:00,18692,1398.0,10.0,10.300429
